# Binance Test Net API Research

In [6]:
import os

import ccxt.pro as pro
import ccxt
import dotenv

In [7]:
dotenv.load_dotenv()
apiKey = os.getenv('TEST_API_KEY2')
apiSecret = os.getenv('TEST_API_SECRET2')

In [9]:
exchange_class = getattr(ccxt, 'binanceus')
exchange = exchange_class({'enableRateLimit': True,
                           'apiKey': apiKey, 'apiSecret': apiSecret})
exchange.set_sandbox_mode(True)

In [14]:
exchange.fetchOrderBook ('BTC/USDT')

({'symbol': 'BTC/USDT',
  'bids': [[39674.62, 0.00048],
   [39674.61, 0.008],
   [39674.6, 0.20894],
   [39671.84, 0.73053],
   [39671.83, 0.49599],
   [39663.34, 0.50585],
   [39663.32, 0.00059],
   [39653.0, 0.00327],
   [39646.2, 0.86243],
   [39640.05, 0.00024],
   [39618.52, 0.00043],
   [39608.72, 0.00357],
   [39608.04, 0.00059],
   [39597.8, 0.00036],
   [39593.19, 0.00033],
   [39592.91, 0.00585],
   [39592.9, 0.86243],
   [39592.3, 2.15452],
   [39585.32, 0.00027],
   [39583.41, 0.07216],
   [39578.95, 0.0025],
   [39571.5, 0.86243],
   [39570.91, 0.00815],
   [39566.87, 0.00358],
   [39566.23, 0.001],
   [39566.01, 0.00015],
   [39565.93, 5e-05],
   [39560.0, 0.001],
   [39552.76, 0.00059],
   [39548.8, 0.00025],
   [39544.68, 0.00033],
   [39541.26, 0.00133],
   [39538.57, 5e-05],
   [39538.45, 0.00067],
   [39535.41, 0.00432],
   [39534.36, 0.00076],
   [39531.88, 0.00087],
   [39531.78, 0.00011],
   [39529.66, 0.00358],
   [39523.58, 0.00024],
   [39520.69, 0.00044],
   [